안녕하세요.  
"나만의 AI챗봇 만들기" 프로젝트를 발표할 이연준입니다.🚀  

2022년 말 인공지능의 특이점이라고 불릴 수도 있는 "챗GPT"가 등장하였습니다.  
챗GPT는 학계, 산업 등 다양한 분야에서 놀라운 기록을 세우고 있습니다.  
챗GPT의 등장으로 생성형 인공지능 언어모델을 이해하고 잘 활용하는 능력이 정말 중요해 졌습니다.🚀  

이 같은 배경에서 본 프로젝트는 챗GPT까지는 아니어도 낮은 수준의 인공지능 언어모델을 만들기 위하여 다음과 같은 프로젝트를 준비하였습니다.  

먼저 본 프로젝트의 목표는 챗GPT의 기본모델인 트렌스포머모델을 사용하여 일상대화가 가능한 수준의 챗봇을 만드는 것입니다.  
그리고 "나만의 챗봇"이기 때문에 챗봇 스스로가 자신을 "이연준의 챗봇"이라고 인식하게 하는 것이 본 프로젝트의 목표입니다.  
챗봇이 학습할 데이터에 "이연준의 챗봇"이라는 데이터를 학습시킴으로써 챗봇 스스로가 "이연준의 챗봇"이라 인식할 수 있을 것으로 기대 됩니다.🚀  

1. 데이터 준비  

나만의 챗봇을 만들기 위하여 "감성대화 말뭉치" 데이터를 사용하겠습니다.🚀  

감성대화 말뭉치는 AI 언어모델을 만들기 위하여 수집된 데이터입니다.  
5개월이 넘는 기간동안 일반인 1500명을 대상으로 수집되었고, 60가지의 감정 상태가 포함되어있습니다.  
감성대화 말뭉치는 인공지능 언어모델을 개발하기 편리하도록 잘 정제되어있어서, 일상대화를 할 수 있는 챗봇을 만들기 위한 적절한 데이터라고 여겨집니다.🚀  

감성대화 말뭉치 데이터 안에는 총 58,268개 말뭉치 데이터가 있습니다.  
하나의 말뭉치 데이터에는 질문과 답변이 3쌍씩 들어있습니다.    
본 프로젝트에서는 간단한 일상대화를 하는 챗봇을 만들 것이기 때문에 첫번째 턴만 사용하도록 하겠습니다.🚀  

본 프로젝트에서는 스스로를 "이연준의 챗봇"이라 인식하는 챗봇을 만들것입니다.  
그래서 훈련데이터에 이와 관련된 데이터를 다음과 같이 추가해주었습니다.🚀  

본 프로젝트에서는 사용하기 편리한 서브워드텍스트인코더를 활용하여 토큰화를 진행하였습니다.  
서브워드텍스트인코더는 텐서플로우 라이브러리를 통하여 쉽게 사용이 가능하고, 미등록단어는 학습이 어려워지는 문제를 어느정도 해결할 수 있습니다.🚀    

본 프로젝트에서 사용할 트랜스포머 모델은 3가지 종류의 데이터가 필요합니다.  
질문 데이터는 START 토큰과 END 토큰이 포함되어 최초 인코더에 입력됩니다.  
그리고 답변 데이터는 두가지로 구분되는데, START 토큰이 있는 답변 데이터는 디코더에 입력되고, END 토큰만 있는 답변 데이터는 예측타겟으로 사용됩니다.🚀  



2. 모델 구성  

본 프로젝트의 트랜스포머 모델은 Attention Is All You Need 논문을 배경으로 TensorFlow의 라이브러리를 적극 활용하여 만들어졌습니다.🚀  

트랜스포머 모델은 위치인코딩이 포함된 임베딩으로 시작합니다.  
이미 토큰화가 된 입력 데이터를 임베딩벡터로 변환하고, 변환된 임베딩 벡터에 사인, 코사인 함수를 적용하여 위치정보를 생성합니다.🚀  
생성된 위치 정보를 시각화하면 다음과 같습니다.  
그래프에서 가로축은 본 프로젝트에서 사용하는 약10,000개의 단어집합을 의미하고, 세로축은 임베딩벡터의 차원을 의미합니다.  
정리하면 단어집합과 임베딩벡터 차원에 대하여 싸인, 코싸인 함수를 적용하여 -1에서 1까지의 값을 할당한 것이라고 볼 수 있습니다.🚀  

인코더의 첫번째 층은 글로벌 셀프 어텐션 레이어입니다.  
이곳에서는 입력된 데이터에 대하여 쿼리, 키, 벨류 연산을 수행하고, 입력값과 출력값을 합치고 정규화하는 과정을 거칩니다.🚀  

다음으로 디코더의 첫번째 층은 캐주얼 셀프 어텐션 레이어입니다.  
이곳에서도 역시 입력된 데이터에 대하여 쿼리, 키, 벨류 연산을 수행하는데, 단 답변에 대한 예측을 위하여 마스킹 과정이 포함 됩니다.🚀  

다음은 크로스 어텐션 레이어 입니다.  
이곳에서도 역시 쿼리, 키, 벨류 연산이 수행되는데, 이때 쿼리는 디코더의 입력 데이터이고, 키, 벨류는 인코더의 출력데이터입니다.🚀  

학습 데이터의 질문과 답변 데이터를 통하여 각 단어의 출현 확률을 추정하고 학습합니다.  
그리고 [END]토큰이 추정 될 때까지 단어를 예측하게 됩니다.🚀  


3. 모델 학습 및 평가🚀  

모델의 학습률은 학습진행 과정에 따라 학습률이 조정되는 메커니즘을 사용하였습니다.  
모델을 경량화하여 레이어와 임베딩차원 등을 지정하였고, 과적합방지를 위한 Drop Out를 적용하였습니다.🚀  

모델 학습을 진행한 결과 4 에포크가 넘어가면 과적합이 발생하고, 오히려 검증데이터의 손실이 늘어나는 것을 볼 수 있습니다.🚀  

학습된 모델을 평가하기 위하여 몇가지 질문을 하였습니다.  
첫번째 질문은 요즘 저의 걱정을 챗봇에게 말해보았습니다.    
그러자 저의 챗봇이 "걱정이 많으시겠어요 .  미래에 대한 불안이 크면 어떻게 하면 좋을까요 ?"라고 저의 고민에 적절하게 답변해 주었습니다.🚀  
두번째 질문은 제가 지난 생일에 딸에게 받은 생일선물에 대한 기쁨을 챗봇에게 말해보았습니다.  
그러자 저의 챗봇이 "따님분이 선물을 주셨군요 .  정말 행복하시겠어요 ."라고 긍정적인 감정에 대해서도 적절하게 답변해 주었습니다.🚀  
세번째, 챗봇이 스스로 누구라고 인식하는지 질문해 보았습니다.  
나만의 챗봇에 "너는 누구니?"라고 질문하였더니, 모델에서 학습한 대로 "저는 이연준님이 만든 챗봇이에요"라고 답변하였습니다.🚀  
마지막으로 챗봇에게 이해하기 힘든 말을 해보았습니다.  
나만의 챗봇에 "아무리 생각해도 너는 멍청한 것 같아."라고 입력하였더니, "무엇 때문에 말에 상처받아 할까요 ?"라고 이해하기 힘든 답변을 하였습니다.🚀  



4. 결론 및 한계점🚀  

먼저 본 프로젝트의 첫번째 목표인 "일상대화가 가능한 수준의 챗봇" 만들기는 절반의 성공에 머물렀다고 평가할 수 있습니다.  
본 프로젝트의 챗봇은 매우 낮은 수준의 일상대화만 가능하였고, 대화의 맥락은 전혀 이해하지 못하였습니다.  
또한 챗봇의 학습과정에서 "감성대화 말뭉치"를 사용하여 엄밀한 의미의 일상대화가 아닌 "감성적인 답변"만 하는 챗봇이라는 한계가 있었습니다.  

다음으로 챗봇 스스로 "이연준의 챗봇"이라고 인식하도록 하는 목표는 어느정도 달성된 것으로 평가할 수 있습니다.  
챗봇의 학습과정에서 "이연준의 챗봇"이라 인식하는 데이터를 학습시킴으로써 챗봇 스스로가 "이연준의 챗봇"이라 인식할 수 있게 되었습니다.  

마지막으로 본 프로젝트의 모델의 복잡도를 증가시키면 오히려 학습이 잘 안되는 문제가 발생하였습니다.  
이와 관련해서는 모델의 학습 과정에 대한 정확한 이해를 바탕으로 과적합 방지를 위한 대처방안과 역전파시 가중치 업데이트에 대한 개선이 필요할 것으로 예상됩니다.🚀

이상으로 "나만의 AI챗봇 만들기" 프로젝트에 대한 발표를 모두 마치겠습니다.  
감사합니다.
